In [1]:
from dataset import get_dataloaders
train_loader_labeled, val_loader_labeled, train_loader_unlabeled = get_dataloaders('data3/mendeley', 16)

In [2]:
from convnext import Autoencoder
import torch
import torch.nn as nn
import torch.optim as optim


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Autoencoder(11).to(DEVICE)

In [3]:
from train import train_model, validate_model

model = train_model(
    model,
    train_loader_labeled,
    train_loader_unlabeled,
    lr=1e-4,
    num_epochs=35,
)
# Epoch 1/35:  16%|█▌        | 54/337 [01:35<07:59,  1.69s/it]

Epoch [1/35], Total Loss: 709.1469, Reconstruction Loss: 27.9685, Classification Loss: 681.1784, Accuracy: 0.2621


Epoch [2/35], Total Loss: 650.0960, Reconstruction Loss: 26.9232, Classification Loss: 623.1727, Accuracy: 0.3323


Epoch [3/35], Total Loss: 572.0640, Reconstruction Loss: 26.3723, Classification Loss: 545.6917, Accuracy: 0.4062


Epoch [4/35], Total Loss: 463.2746, Reconstruction Loss: 25.9408, Classification Loss: 437.3338, Accuracy: 0.5447


Epoch [5/35], Total Loss: 342.8847, Reconstruction Loss: 25.5847, Classification Loss: 317.3000, Accuracy: 0.6654


...

Epoch [31/35], Total Loss: 27.5699, Reconstruction Loss: 23.9670, Classification Loss: 3.6030, Accuracy: 0.9950


Epoch [32/35], Total Loss: 28.6876, Reconstruction Loss: 23.9773, Classification Loss: 4.7103, Accuracy: 0.9937


Epoch [33/35], Total Loss: 57.9209, Reconstruction Loss: 24.0297, Classification Loss: 33.8912, Accuracy: 0.9694


Epoch [34/35], Total Loss: 30.7392, Reconstruction Loss: 23.9568, Classification Loss: 6.7824, Accuracy: 0.9922


Epoch [35/35], Total Loss: 27.5869, Reconstruction Loss: 23.9448, Classification Loss: 3.6421, Accuracy: 0.9955


In [4]:
torch.save(model.state_dict(), 'models/convnext_mendeley.pth')

In [5]:
validate_model(model, val_loader_labeled, 11)
# Validation Accuracy: 0.6914
# Validation Precision: 0.7020
# Validation Recall: 0.6241
# Validation F1 Score: 0.6460
# Number of samples validated on: 256

Validation Accuracy: 0.6852
Validation Precision: 0.6702
Validation Recall: 0.6274
Validation F1 Score: 0.6399
Number of samples validated on: 270
